In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.training import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(0)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [ ]:
#TEST NOTES:  Replacing batchnorm with instance norm; Adding "shock absorbing" training sessions between size changes.
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/train')
OPENIMAGES = Path('data/openimages')
CIFAR10 = Path('data/cifar10/train')

proj_id = 'super_res'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

#gpath = IMAGENET.parent/('bwc_withattn_sn_supertrain2x_gen_96.h5')
#dpath = IMAGENET.parent/('bwc_withattn_sn_supertrain2x_critic_96.h5')

c_lr=1e-3
c_lrs = np.array([c_lr,c_lr,c_lr])

g_lr=c_lr/5
g_lrs = np.array([g_lr/1000,g_lr/100,g_lr])

keep_pcts=[0.20,0.20]
gen_freeze_tos=[-1,0]

scale=2
lrs_unfreeze_factor=1.0
x_tfms=[]
extra_aug_tfms=[]

torch.backends.cudnn.benchmark=True

## Training

In [ ]:
netG = Unet34(nf_factor=1, scale=scale).cuda()
#netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = DCCritic(ni=3, nf=128).cuda()
#netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [ ]:
trainer = GANTrainer(netD=netD, netG=netG, genloss_fns=[FeatureLoss(multiplier=1e2)])
trainerVis = GANVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer', jupyter=False, visual_iters=100)

In [ ]:
scheds=[]

scheds.extend(GANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=keep_pcts, 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos, reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[128,128], bss=[32,32], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3,0.3], 
    save_base_name=proj_id, c_lrs=c_lrs, g_lrs=g_lrs, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=gen_freeze_tos, reduce_x_scale=scale))



#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/20, g_lrs=g_lrs/20, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/2, g_lrs=g_lrs/2, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/6, g_lrs=g_lrs/6, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[192], bss=[16], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/6, g_lrs=g_lrs/6, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))


#unshock
scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.1], 
    save_base_name=proj_id, c_lrs=c_lrs/120, g_lrs=g_lrs/120, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/12, g_lrs=g_lrs/12, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/12, g_lrs=g_lrs/12, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[-1], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=IMAGENET, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.2], 
    save_base_name=proj_id, c_lrs=c_lrs/24, g_lrs=g_lrs/24, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))

scheds.extend(GANTrainSchedule.generate_schedules(szs=[256], bss=[8], path=OPENIMAGES, x_tfms=x_tfms, extra_aug_tfms=extra_aug_tfms, keep_pcts=[0.3], 
    save_base_name=proj_id, c_lrs=c_lrs/24, g_lrs=g_lrs/24, lrs_unfreeze_factor=lrs_unfreeze_factor, gen_freeze_tos=[0], reduce_x_scale=scale))



In [ ]:
trainer.train(scheds=scheds)